In [ ]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval

In [ ]:
from thirdai.demos import download_beir_dataset

#### dataset and filenames for UDT

dataset = "scifact"
unsup_file, sup_train_file, sup_test_file, n_target_classes = download_beir_dataset(dataset)

In [ ]:
from thirdai import bolt,licensing

licensing.activate('8E46E4-653FD6-AA2B02-65E265-A4FACA-V3')

model = bolt.UniversalDeepTransformer(
    data_types={
        "QUERY": bolt.types.text(),
        "DOC_ID": bolt.types.categorical(delimiter=':'),
    },
    target="DOC_ID",
    n_target_classes=n_target_classes,
    integer_target=True,
    model_config='./configs/embeddings_and_cold_start.config',
)

In [ ]:
model.cold_start(
    filename=unsup_file,
    strong_column_names=["TITLE"],
    weak_column_names=["TEXT"],
    learning_rate=0.001,
    epochs=5,
)

activations = model.evaluate(sup_test_file, metrics=['categorical_accuracy','recall@100'])

In [ ]:
model.train(
    filename=sup_train_file,
    learning_rate=0.001,
    epochs=5,
)

activations = model.evaluate(sup_test_file, metrics=['categorical_accuracy','recall@100'])

In [ ]:
model.save('./scidocs.model')

model = bolt.UniversalDeepTransformer.load('./scidocs.model')